## Data Set Preparation

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("train.csv")

In [3]:
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [4]:
from sklearn.preprocessing import LabelEncoder
sex_encoder = LabelEncoder()
df['Sex'] = sex_encoder.fit_transform(df['Sex'])

Male = 1 and Female = 0

In [5]:
correlation = df.corr()
correlation['Survived'].sort_values(ascending = False)

Survived       1.000000
Fare           0.257307
Parch          0.081629
PassengerId   -0.005007
SibSp         -0.035322
Age           -0.077221
Pclass        -0.338481
Sex           -0.543351
Name: Survived, dtype: float64

Use just some of the basic parameters, which are age, pclass, and sex

In [6]:
parameters = ['Sex', 'Age', 'Pclass']
X = df[parameters]
y = df['Survived']

In [7]:
X = pd.concat([X, pd.get_dummies(X['Pclass'])], axis = 1)
X = X.drop('Pclass', axis = 1)

In [8]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
Sex    891 non-null int64
Age    714 non-null float64
1      891 non-null uint8
2      891 non-null uint8
3      891 non-null uint8
dtypes: float64(1), int64(1), uint8(3)
memory usage: 16.6 KB


In [9]:
X.head(10)

,Sex,Age,1,2,3
0,1,22.0,0,0,1
1,0,38.0,1,0,0
2,0,26.0,0,0,1
3,0,35.0,1,0,0
4,1,35.0,0,0,1
5,1,NaN,0,0,1
6,1,54.0,1,0,0
7,1,2.0,0,0,1
8,0,27.0,0,0,1
9,0,14.0,0,1,0


In [10]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy = 'median')
imputer.fit(X)
imputer.statistics_

array([ 1., 28.,  0.,  0.,  1.])

In [11]:
X_tr = imputer.transform(X)
X = pd.DataFrame(X_tr, columns = X.columns)

In [12]:
X.head(10)

,Sex,Age,1,2,3
0,1.0,22.0,0.0,0.0,1.0
1,0.0,38.0,1.0,0.0,0.0
2,0.0,26.0,0.0,0.0,1.0
3,0.0,35.0,1.0,0.0,0.0
4,1.0,35.0,0.0,0.0,1.0
5,1.0,28.0,0.0,0.0,1.0
6,1.0,54.0,1.0,0.0,0.0
7,1.0,2.0,0.0,0.0,1.0
8,0.0,27.0,0.0,0.0,1.0
9,0.0,14.0,0.0,1.0,0.0


In [13]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X['Age'] = scaler.fit_transform(X['Age'].values.reshape(len(X),1),1)

In [14]:
X.head(10)

,Sex,Age,1,2,3
0,1.0,0.271174,0.0,0.0,1.0
1,0.0,0.472229,1.0,0.0,0.0
2,0.0,0.321438,0.0,0.0,1.0
3,0.0,0.434531,1.0,0.0,0.0
4,1.0,0.434531,0.0,0.0,1.0
5,1.0,0.346569,0.0,0.0,1.0
6,1.0,0.673285,1.0,0.0,0.0
7,1.0,0.019854,0.0,0.0,1.0
8,0.0,0.334004,0.0,0.0,1.0
9,0.0,0.170646,0.0,1.0,0.0


## Creating the Model

In [15]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

Using TensorFlow backend.


In [16]:
model = Sequential()

model.add(Dense(input_dim = 5, units=32, activation='relu'))
model.add(Dropout(rate=0.3))

model.add(Dense(units=16, activation='relu'))

model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.values, y, test_size=0.2)
model.fit(X_train, y_train, batch_size = 10, epochs = 500)

Instructions for updating:
Use tf.cast instead.
Epoch 1/500
712/712 [==============================] - 0s 619us/step - loss: 0.6081 - acc: 0.6489
Epoch 2/500
712/712 [==============================] - 0s 271us/step - loss: 0.5348 - acc: 0.7725
Epoch 3/500
712/712 [==============================] - 0s 230us/step - loss: 0.4860 - acc: 0.7851
Epoch 4/500
712/712 [==============================] - 0s 167us/step - loss: 0.4687 - acc: 0.7753
Epoch 5/500
712/712 [==============================] - 0s 200us/step - loss: 0.4630 - acc: 0.7809
Epoch 6/500
712/712 [==============================] - 0s 230us/step - loss: 0.4679 - acc: 0.7598 0s - loss: 0.4490 - acc: 0.76
Epoch 7/500
712/712 [==============================] - 0s 200us/step - loss: 0.4597 - acc: 0.7851
Epoch 8/500
712/712 [==============================] - 0s 207us/step - loss: 0.4482 - acc: 0.7921
Epoch 9/500
712/712 [==============================] - 0s 175us/step - loss: 0.4513 - acc: 0.7921
Epoch 10/500
712/712 [==================

712/712 [==============================] - 0s 143us/step - loss: 0.4192 - acc: 0.8104
Epoch 81/500
712/712 [==============================] - 0s 144us/step - loss: 0.4188 - acc: 0.8048
Epoch 82/500
712/712 [==============================] - 0s 148us/step - loss: 0.4253 - acc: 0.7935
Epoch 83/500
712/712 [==============================] - 0s 155us/step - loss: 0.4200 - acc: 0.8020
Epoch 84/500
712/712 [==============================] - 0s 154us/step - loss: 0.4139 - acc: 0.8020
Epoch 85/500
712/712 [==============================] - 0s 149us/step - loss: 0.4210 - acc: 0.8006
Epoch 86/500
712/712 [==============================] - 0s 144us/step - loss: 0.4203 - acc: 0.7893
Epoch 87/500
712/712 [==============================] - 0s 152us/step - loss: 0.4227 - acc: 0.8034
Epoch 88/500
712/712 [==============================] - 0s 163us/step - loss: 0.4258 - acc: 0.7879
Epoch 89/500
712/712 [==============================] - 0s 160us/step - loss: 0.4206 - acc: 0.8020
Epoch 90/500
712/712 [=

712/712 [==============================] - 0s 227us/step - loss: 0.4229 - acc: 0.7963
Epoch 163/500
712/712 [==============================] - 0s 202us/step - loss: 0.4097 - acc: 0.8048
Epoch 164/500
712/712 [==============================] - 0s 188us/step - loss: 0.4126 - acc: 0.7963
Epoch 165/500
712/712 [==============================] - 0s 173us/step - loss: 0.4206 - acc: 0.8076
Epoch 166/500
712/712 [==============================] - 0s 190us/step - loss: 0.4164 - acc: 0.8174
Epoch 167/500
712/712 [==============================] - 0s 156us/step - loss: 0.4111 - acc: 0.8118
Epoch 168/500
712/712 [==============================] - 0s 141us/step - loss: 0.4119 - acc: 0.8132
Epoch 169/500
712/712 [==============================] - 0s 151us/step - loss: 0.4170 - acc: 0.8160
Epoch 170/500
712/712 [==============================] - 0s 144us/step - loss: 0.4107 - acc: 0.8048
Epoch 171/500
712/712 [==============================] - 0s 201us/step - loss: 0.4124 - acc: 0.8076
Epoch 172/500


712/712 [==============================] - 0s 139us/step - loss: 0.4079 - acc: 0.8174
Epoch 245/500
712/712 [==============================] - 0s 160us/step - loss: 0.4092 - acc: 0.8160
Epoch 246/500
712/712 [==============================] - 0s 172us/step - loss: 0.4096 - acc: 0.8090
Epoch 247/500
712/712 [==============================] - 0s 151us/step - loss: 0.4138 - acc: 0.8062
Epoch 248/500
712/712 [==============================] - 0s 135us/step - loss: 0.4113 - acc: 0.8090
Epoch 249/500
712/712 [==============================] - 0s 135us/step - loss: 0.4073 - acc: 0.8118
Epoch 250/500
712/712 [==============================] - 0s 134us/step - loss: 0.4116 - acc: 0.8118
Epoch 251/500
712/712 [==============================] - 0s 131us/step - loss: 0.4090 - acc: 0.8132
Epoch 252/500
712/712 [==============================] - 0s 127us/step - loss: 0.4119 - acc: 0.8258
Epoch 253/500
712/712 [==============================] - 0s 124us/step - loss: 0.4099 - acc: 0.8062
Epoch 254/500


712/712 [==============================] - 0s 156us/step - loss: 0.4068 - acc: 0.8174
Epoch 327/500
712/712 [==============================] - 0s 234us/step - loss: 0.4106 - acc: 0.8076
Epoch 328/500
712/712 [==============================] - 0s 192us/step - loss: 0.4013 - acc: 0.8160
Epoch 329/500
712/712 [==============================] - 0s 195us/step - loss: 0.4102 - acc: 0.8118
Epoch 330/500
712/712 [==============================] - 0s 269us/step - loss: 0.4067 - acc: 0.8174
Epoch 331/500
712/712 [==============================] - 0s 223us/step - loss: 0.4003 - acc: 0.8160
Epoch 332/500
712/712 [==============================] - 0s 139us/step - loss: 0.3982 - acc: 0.8188
Epoch 333/500
712/712 [==============================] - 0s 141us/step - loss: 0.4119 - acc: 0.8048
Epoch 334/500
712/712 [==============================] - 0s 143us/step - loss: 0.4081 - acc: 0.8062
Epoch 335/500
712/712 [==============================] - 0s 171us/step - loss: 0.4049 - acc: 0.8090
Epoch 336/500


712/712 [==============================] - 0s 143us/step - loss: 0.4049 - acc: 0.8160
Epoch 409/500
712/712 [==============================] - 0s 144us/step - loss: 0.4071 - acc: 0.8090
Epoch 410/500
712/712 [==============================] - 0s 167us/step - loss: 0.4098 - acc: 0.8104
Epoch 411/500
712/712 [==============================] - 0s 146us/step - loss: 0.4050 - acc: 0.8230
Epoch 412/500
712/712 [==============================] - 0s 151us/step - loss: 0.4019 - acc: 0.8216
Epoch 413/500
712/712 [==============================] - 0s 152us/step - loss: 0.4041 - acc: 0.8160
Epoch 414/500
712/712 [==============================] - 0s 150us/step - loss: 0.4020 - acc: 0.8188
Epoch 415/500
712/712 [==============================] - 0s 150us/step - loss: 0.4050 - acc: 0.8160
Epoch 416/500
712/712 [==============================] - 0s 147us/step - loss: 0.4049 - acc: 0.8174
Epoch 417/500
712/712 [==============================] - 0s 180us/step - loss: 0.4021 - acc: 0.8146
Epoch 418/500


712/712 [==============================] - 0s 178us/step - loss: 0.3995 - acc: 0.8132
Epoch 491/500
712/712 [==============================] - 0s 156us/step - loss: 0.4019 - acc: 0.8090
Epoch 492/500
712/712 [==============================] - 0s 136us/step - loss: 0.3995 - acc: 0.8118
Epoch 493/500
712/712 [==============================] - 0s 136us/step - loss: 0.4027 - acc: 0.8146
Epoch 494/500
712/712 [==============================] - 0s 140us/step - loss: 0.4018 - acc: 0.8146
Epoch 495/500
712/712 [==============================] - 0s 147us/step - loss: 0.4076 - acc: 0.8118
Epoch 496/500
712/712 [==============================] - 0s 168us/step - loss: 0.4062 - acc: 0.8118
Epoch 497/500
712/712 [==============================] - 0s 149us/step - loss: 0.4027 - acc: 0.8160
Epoch 498/500
712/712 [==============================] - 0s 148us/step - loss: 0.4032 - acc: 0.8230
Epoch 499/500
712/712 [==============================] - 0s 152us/step - loss: 0.4073 - acc: 0.8146
Epoch 500/500


## Making Predictions

In [18]:
predictions = model.predict(X_test)
predictions[predictions>=0.6] = 1
predictions[predictions<0.6] = 0

In [19]:
def accuracy_predictor(predictions, test_set, original_labels):
    correct = 0
    for i in range(len(test_set)):
        if predictions[i] == original_labels.iloc[i]:
            correct+=1
    accuracy = correct/len(test_set)
    return accuracy

In [20]:
accuracy_predictor(predictions=predictions, test_set=X_test, original_labels=y_test)

0.7877094972067039

In [21]:
def survival_probability(sex, age, pclass1, pclass2, pclass3):
    age = np.arange(age, age+1, 1)
    user = [sex, scaler.transform(age.reshape(-1,1)), pclass1, pclass2, pclass3]
    survived = model.predict(np.array(user).reshape(1,5))
    return survived[0][0]*100

In [22]:
my_survival_chances = survival_probability(sex = 1, age = 20, pclass1 = 1, pclass2 = 0, pclass3 = 0)
my_survival_chances

40.9126877784729